# Word2Vec

## Imports

In [ ]:
import pandas as pd
import numpy as np
import gensim
import gensim.downloader
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelBinarizer

import re
from bs4 import BeautifulSoup

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

## Un modelo Word2Vec pre-entrenado

In [ ]:
print(list(gensim.downloader.info()['models'].keys()))

In [ ]:
# Cargamos el modelo pre-entrenado
w2v = gensim.downloader.load('glove-wiki-gigaword-50')

In [ ]:
# Primeros 10 tokens del vocabulario
w2v.index_to_key[:10]

In [ ]:
# Total de tokens en el vocabulario
len(w2v.index_to_key)

In [ ]:
# Dimensión del embedding
w2v['play'].shape

In [ ]:
# Devuelve lista de las 5 palabras más similares a 'king' y su similitud.
similar_words = w2v.most_similar('king', topn=5)
print(similar_words)

In [ ]:
# Esto debería devolver algo cercano a 'queen'.
result = w2v.most_similar(positive=['king', 'woman'], negative=['man'], topn=1)
print(result)

In [ ]:
# Esto devuelve un valor entre -1 y 1, donde 1 indica que son muy similares.
similarity = w2v.similarity('king', 'queen')
print(similarity)

In [ ]:
# Esto debería devolver 'car', ya que no es una comida.
odd_one_out = w2v.doesnt_match(['breakfast', 'lunch', 'dinner', 'car'])
print(odd_one_out)

In [ ]:
# Selecciona las palabras que quieres visualizar
words = ['king', 'queen', 'man', 'woman']

# Extrae los vectores de esas palabras
word_vectors = [w2v[word] for word in words]

# Aplica PCA para reducir la dimensionalidad a 2D
pca = PCA(n_components=2)
word_vecs_2d = pca.fit_transform(word_vectors)

# Graficar los resultados
plt.figure(figsize=(4, 3))
for i, word in enumerate(words):
    plt.scatter(word_vecs_2d[i][0], word_vecs_2d[i][1])
    plt.annotate(word, (word_vecs_2d[i][0], word_vecs_2d[i][1]))

plt.title("Visualización de Word2Vec")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.show()

## Embedding de documentos

Vamos a usar el dataset de IMDB para clasificación de reseñas de películas, el objetivo del mismo es detectar si una reseña tiene sentimiento **positivo** o **negativo**.

In [ ]:
imdb_data = pd.read_csv("./IMDB Dataset.csv")

In [ ]:
imdb_data.head()

In [ ]:
imdb_data['sentiment'].value_counts()

In [ ]:
# Convert positive and negative into binary classes (1-0)
lb = LabelBinarizer()

sentiment_data = lb.fit_transform(imdb_data["sentiment"])
imdb_data['sentiment'] = sentiment_data

In [ ]:
imdb_data.head()

Primero haremos:
1.   Eliminar tags html (vamos a utilizar BeautifulSoup para esto)
2.   Eliminar texto entre parentesis rectos (Usando la siguiente expresion regular: ```\[[^]]*\]``` )
3. Eliminar caracteres especiales, usando una regex quitar todos los caracteres que no son ni letras ni números (```[^a-zA-z0-9\s] ``` )

In [ ]:
def strip_html(text):
  soup = BeautifulSoup(text, 'html.parser')
  return soup.get_text()

def remove_between_square_brackets(text):
  out = re.sub('\[[^]]*\]','',text)
  return out

def remove_special_characters(text):
  out = re.sub('[^a-zA-Z0-9\s]','',text)
  return out

def low_level_preproc(text):
  return remove_special_characters(remove_between_square_brackets(strip_html(text)))

In [ ]:
imdb_data['review'] = imdb_data['review'].apply(low_level_preproc)

Una vez tenemos el texto limpio y trabajable volvemos a hacer otro pasaje de preprocesamiento de más alto nivel, ahora vamos a querer:

1.   Transformar todo el texto a minúscula
2.   Quitar stop words (usando nltk)
3.   Lemmatizar usando nltk WordNetLemmatizer

Para todo esto vamos a necesitar trabajar con **tokens** palabras individuales, en este caso vamos a separar por **whitespace**, pero se podrían usar mejores estrategias.

In [ ]:
all_stopwords = set(stopwords.words("english"))

def remove_stop_words(full_text_line):
  tokens = full_text_line.split(" ")
  result = [token for token in tokens if token not in all_stopwords]
  return result

def lemmatize(tokens):
  lemmatizer = WordNetLemmatizer()
  result = [lemmatizer.lemmatize(token) for token in tokens]
  return result

def high_level_preproc(text):
  result = remove_stop_words(text)
  result = lemmatize(result)
  return " ".join(result)

In [ ]:
imdb_data['review'] = imdb_data['review'].str.lower()
imdb_data['review'] = imdb_data['review'].apply(high_level_preproc)

In [ ]:
imdb_data['review'].head()

In [ ]:
mean_vector = np.mean(w2v.vectors, axis=0)

In [ ]:
def get_sentence_embedding(text):
  tokens = text.split(" ")
  embeddings = [w2v[token] if token in w2v else mean_vector for token in tokens]
  return np.mean(np.array(embeddings), axis = 0)

In [ ]:
review_vectors = [get_sentence_embedding(sent) for sent in imdb_data.review]

In [ ]:
# Aplica PCA para reducir la dimensionalidad a 2D
review_vectors = np.array(review_vectors)
pca = PCA(n_components=2)
review_vectors_2d = pca.fit_transform(review_vectors)

# Graficar los resultados
plt.figure(figsize=(8, 6))

# Definir los colores para cada clase 0 y 1
colors = ['tab:red' if sentiment == 0 else 'tab:blue' for sentiment in imdb_data['sentiment']]

# Hacer el scatter plot
scatter = plt.scatter(review_vectors_2d[:, 0], review_vectors_2d[:, 1], c=colors)

# Añadir el título y etiquetas de ejes
plt.title("Visualización de Word2Vec con PCA")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")

# Añadir leyenda manualmente para 0 y 1
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor='tab:red', markersize=10, label='Sentimiento 0'),
                   Line2D([0], [0], marker='o', color='w', markerfacecolor='tab:blue', markersize=10, label='Sentimiento 1')]
plt.legend(handles=legend_elements)

# Mostrar el gráfico
plt.show()

### Crear dataframe

Si hubieran más columnas en el dataset original, lo concatenamos luego de crear el dataframe con los embeddings.

In [ ]:
embedding_df = pd.DataFrame(review_vectors)
embedding_df.head()

## Data Split

In [ ]:
from sklearn.model_selection import train_test_split
random_state = 17

X, y = embedding_df, imdb_data["sentiment"]
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=(1.0/3),
    random_state=random_state)

## Modelo de Clasificación

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

y_train_pred = log_reg.predict(X_train)
print(f"Train accuracy: {accuracy_score(y_train, y_train_pred)}")

y_pred = log_reg.predict(X_test)
print(f"Test accuracy: {accuracy_score(y_test, y_pred)}")